In [8]:
from IPython.display import display
from utils import close_notebook, AppWidgets

In [9]:
# Global variables
SAMPLES_COLLECTION_OPENBIS_PATH = "/MATERIALS/SAMPLES/SAMPLE_COLLECTION"

# Widgets
app_widgets = AppWidgets("create_sample", 
                         samples_collection_openbis_path = SAMPLES_COLLECTION_OPENBIS_PATH)

# Connect with openBIS
app_widgets.connect_openbis()

# Create a sample object in openBIS

## Select material

In [ ]:
app_widgets.material_selection_radio_button.observe(app_widgets.select_material_radio_change, names='value')
display(app_widgets.material_selection_radio_button)

## Sample out

In [ ]:
app_widgets.create_button.on_click(app_widgets.create_sample_action)
app_widgets.quit_button.on_click(close_notebook)
display(app_widgets.sample_out_name_textbox, app_widgets.bottom_buttons_hbox)
display(app_widgets.increase_buttons_size)